#### Выводы по анализу данных

Логика в данных та же, что и в задаче 1.

Одно отличие - целевой признак. Соотношение уволившихся сотрудников к сохраненным = 1/2.5

In [2]:
(12-96)/6

-14.0

In [ ]:
'<25'